In [1]:
import numpy as np
import copy

with open("input.txt", 'r') as f:
    input = f.read()

# big example
input1 = """##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""

# small example 
input1 = """########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<"""

# small example fro part 2, works for part 2
input1 = """#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^"""

def preprocess(input: str):
    warehouse, moves = tuple(input.split("\n\n"))
    moves = "".join(moves.split("\n"))
    warehouse = np.array([list(row) for row in warehouse.split("\n")])
    return warehouse, moves

warehouse, moves = preprocess(input)
robo_position = np.where(warehouse == '@')
robo_position = int(robo_position[0][0]), int(robo_position[1][0])
robo_position

(24, 24)

In [2]:
DIRECTIONS = {
    "^": (-1, 0),
    ">": (0, 1),
    "v": (1,0),
    "<": (0,-1)
}
    
def print_warehouse(warehouse):
    print("\n".join(["".join(row.tolist()) for row in warehouse]))

def move(warehouse: np.ndarray, pos: tuple, direction: str):
    dir: tuple = DIRECTIONS[direction]
    new_pos = (pos[0] + dir[0], pos[1] + dir[1])
    # print(f"Move is {direction}, next tile is {warehouse[new_pos]}")
    if warehouse[new_pos] == "#": # stuck
        return warehouse, pos
    if warehouse[new_pos] == ".":
        warehouse[pos], warehouse[new_pos] = ".", "@"
        return warehouse, new_pos
    if warehouse[new_pos] == "O":
        will_move, shift = push(warehouse, pos, dir, shift=1)
        if not will_move:
            return warehouse, pos
        terminal_pos = (pos[0] + (shift+1)*dir[0], pos[1] + (shift+1)*dir[1])
        warehouse[pos], warehouse[new_pos], warehouse[terminal_pos] = ".", "@", "O"
        return warehouse, new_pos
        

def push(warehouse, pos, dir: tuple, shift: int) -> tuple[bool, int]:
    """
    returns whether the move is possible, and how many steps to shift
    """
    new_pos = (pos[0] + (shift+1)*dir[0], pos[1] + (shift+1)*dir[1])
    # print(f"push: {pos}, {shift}, {new_pos}, there exists: {warehouse[new_pos]}")
    if warehouse[new_pos] == "#":
        return False, shift
    if warehouse[new_pos] == ".":
        return True, shift
    if warehouse[new_pos] == "O":
        return push(warehouse, pos, dir, shift+1)
    

def all_moves(warehouse, pos, moves):
    wh = copy.copy(warehouse)
    for m in moves:
        wh, pos = move(wh, pos, m)
    return wh

def sumGPS(warehouse, char):
    sum_gps = 0
    for i in range(warehouse.shape[0]):
        for j in range(warehouse.shape[1]):
            if warehouse[i,j] == char:
                sum_gps += 100*i + j
    return sum_gps


warehouse_final  = all_moves(warehouse, robo_position, moves)
sumGPS(warehouse_final, "O")

1526018

### Part 2

In [3]:
def width2x(warehouse):
    warehouse_wide = np.zeros((warehouse.shape[0], warehouse.shape[1]*2), dtype=str)
    for i in range(warehouse.shape[0]):
        for j in range(warehouse.shape[1]):
            if warehouse[i,j] == ".":
                warehouse_wide[i, 2*j], warehouse_wide[i,2*j+1] = ".", "."
            if warehouse[i,j] == "@":
                warehouse_wide[i, 2*j], warehouse_wide[i,2*j+1] = "@", "."
                robo_position_wide = (i,2*j)
            if warehouse[i,j] == "#":
                warehouse_wide[i, 2*j], warehouse_wide[i,2*j+1] = "#", "#"
            if warehouse[i,j] == "O":
                warehouse_wide[i, 2*j], warehouse_wide[i,2*j+1] = "[]", "]"
    return warehouse_wide, robo_position_wide

warehouse_wide, robo_position_wide = width2x(warehouse)
print_warehouse(warehouse_wide)

####################################################################################################
##..........[][]....[]....[]..[]....##........[][][]....[]......[][]..........[]........[]........##
##..[]##......##[][]..........[]..[]..[]........##....[][]....[][]..####..##............[]##..##..##
##..........[]..........................[]....[]........[][]##......[]....[]..............[]......##
##......[]....##....................##........[]....##......[]..[][][]........[][]................##
##[]..####....[][]........##[]....[][]##....##..##......[][]..[]........[]##....[]..[]..[]....[][]##
##............##....[]....[]##..[][][][]..[]..[][]......[]##..[][][]......##......##........[][][]##
##..##............[]..[][]............[][]..[]..[]........##..........[]..[]..[]..[][]..####[][]..##
##..........[][]....[][][]......[]................[]......[][]..[]........[]..[]..........[]......##
##....[]##......[]##..........[][]......[][]..[][]..[][]....................[][][]..##..##.

In [4]:
def move2(warehouse: np.ndarray, pos: tuple, direction: str):
    dir: tuple = DIRECTIONS[direction]
    new_pos = (pos[0] + dir[0], pos[1] + dir[1])
    if warehouse[new_pos] == "#": # stuck
        return warehouse, pos
    if warehouse[new_pos] == ".":
        warehouse[pos], warehouse[new_pos] = ".", "@"
        return warehouse, new_pos
    if warehouse[new_pos] in ["[", "]"]:
        return push2(warehouse, pos, dir)
        
        
def get_remaining_box(mark, pos):
    if mark == "[": return (pos[0], pos[1]+1)
    if mark == "]": return (pos[0], pos[1]-1)

def push2(warehouse, old_robo_pos, dir: tuple) -> tuple[bool, int]:
    """
    returns the new warehouse after shifting
    """
    new_warehouse = copy.copy(warehouse)
    newly_written  = []
    new_robo_pos = (old_robo_pos[0] + dir[0], old_robo_pos[1] + dir[1])
    affected = [new_robo_pos, get_remaining_box(warehouse[new_robo_pos], new_robo_pos)]
    new_warehouse[old_robo_pos], new_warehouse[new_robo_pos] = ".", "@"
    while len(affected) > 0:
        new_affected = []
        # check for failure first
        for ap in affected:
            new_pos = (ap[0]+dir[0], ap[1]+dir[1])
            if warehouse[new_pos] == "#":
                return warehouse, old_robo_pos
        for ap in affected:
            new_pos = (ap[0]+dir[0], ap[1]+dir[1])
            if warehouse[new_pos] in ["[", "]"]:
                new_affected.extend([new_pos, get_remaining_box(warehouse[new_pos], new_pos)])
            new_warehouse[new_pos] = warehouse[ap]
            newly_written.append(new_pos)
            antipos = ap[0]-dir[0], ap[1]-dir[1]
            if warehouse[antipos] not in ["[", "]", "@"] or (warehouse[antipos] in ["[", "]"] and (ap not in newly_written)):
                new_warehouse[ap] = "."
        new_affected = list(set(new_affected).difference(set(affected)))
        affected = new_affected
    return new_warehouse, new_robo_pos

def all_moves2(warehouse, pos, moves):
    wh = copy.copy(warehouse)
    for m in moves:
        wh, pos = move2(wh, pos, m)
    return wh



print(moves)
warehouse_final = all_moves2(warehouse_wide, robo_position_wide, moves)
sumGPS(warehouse_final, "[")

v^^<<<<<^^>v^>><v^<^<v><^^v<^^<>>^^^<^^>v>>>v>v>>^^<^>>^>><^<<v^^<^v><v^><><v^^<<^>v^v^v>v<<vv<^^<vvv>>>v<>v>v<^>>v<>^><vv>^^vv<v<^^vv^v^<v<>>vvv>vv>>v<^v<v<><<^>^<vv^<^>vv>vv<><>vvvv>^v^^<v>vv<^<v>><vv><>><>>v>^v^^v><v>>v>^<vv^v>v<v^>>^><<<<><^>v><v<>>v>^^^><v>^>v<v^>>^^<vv^>^><^>v>><v^>^>><v<^>>>^^<<>^v<>>^>><v<<>^^><<><^>><<>v<^^v>v^<><^><<^v^^>^>vv^^<^^v<v^v><<v<>^vv<<^<>^v^v^<^^v<^^v^<<v>>v>vv<^<>v>v<^^><v<>>^>v^^^^><>>v>^<^>^^^v^>>>vv>^<vv>^<>^<v<v<vv>>v><>>^<v^^^v>>v<<v>^v<v^<>>v<^v<<vvv<^v>v^^><<^^<v^><^v<<>>v><v^<>>^v^^^^^><>><^^^v<^v><v<>>^vv^^<>v<<<<^vv^<<>^^<<>vv^vv<v^<^v<>><<v<v<<^v>>^>vvv>>>^vv<<<v<>v^>^>vv^>v<v<<><v^<^v<>v><<^<v<^<^^v^vvvv^>^v^>vv<v>^v><<vv^<v<<^v>>vv>v<><v^>^<^><^><><^<^<<>^v^^<<v^>^^><><<>v^><>vv<<v^<<vvv>>^^>><>v><^vv<v>>vv<^><v<v^^v>>><v^vv^vvv^>v>v^^<>^^>><^><^v>^>v<<v<^>vv>^<v^<^^vvvv><<v^<^^><v^>^^<^vv>^<v>>v><<^v<v<<>>^><<v^vv<>^v><>^<<>^<<><>v^<><<<^>><>^><v>>>>><^<^<vvvvvv>vv>>vv>^>><v><^<<>>vv^v<^>^^^^v<<v>>>v<<^<<<<v<^vv>^<>><v>^<^v>>v><<>>^v

1550677